## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from potentiel_solaire.constants import RESULTS_FOLDER, DATA_FOLDER
from potentiel_solaire.sources.utils import download_file

## Récupération des résultats des deux methodes

In [ ]:
results_with_simplified_method = gpd.read_file(RESULTS_FOLDER / "priotirized_schools_buildings.gpkg", layer="results_with_simplified_method")
results_with_segmentation = pd.read_csv(RESULTS_FOLDER / "roof_segments_potential.csv").rename(columns={"surface": "roof_surface"})

# Identification des toits plats
results_with_segmentation["is_flat"] = results_with_segmentation["slope_bin_min"] == 0

# Calcule de la surface de toit plat
results_with_segmentation["flat_roof_surface"] = results_with_segmentation["roof_surface"].where(
    results_with_segmentation["is_flat"], 0
)

# Calcul de la surface utile (methode par segmentation) en utilisant la meme definition que IDF
# Surface présentant une irradiation suffisante > 900 kWh/m².an et n’ayant pas d’obstacle (cheminée, velux, aération…)
results_with_segmentation["utility_surface"] = results_with_segmentation["roof_surface"].where(
    results_with_segmentation["solar_irradiation"] > 900, 0
)

# Calcul du potentiel solaire en ne prenant que les segments avec surface utile > 0
results_with_segmentation["solar_potential"] = results_with_segmentation["solar_potential"].where(
    results_with_segmentation["utility_surface"] > 0, 0
)

# Synthese des resultats de la methode avec segmentation au niveau des batiments
results_with_segmented_roofs = results_with_segmentation.groupby(by="cleabs_bat").agg(
    roof_surface=("roof_surface", "sum"),  # methode avec segmentation (MNS)
    flat_roof_surface=("flat_roof_surface", "sum"),  # methode avec segmentation (MNS)
    utility_surface=("utility_surface", "sum"),  # methode avec segmentation (MNS)
    solar_potential=("solar_potential", "sum"),  # methode avec segmentation (MNS)
).reset_index()

# Comparaison des resultats entre les deux methodes
results_comparison_buildings = pd.merge(
    results_with_simplified_method[["code_region", "code_departement", "identifiant_de_l_etablissement", "cleabs_bat", "surface_totale_au_sol", "surface_utile", "potentiel_solaire"]],
    results_with_segmented_roofs[["cleabs_bat", "roof_surface", "flat_roof_surface", "utility_surface", "solar_potential"]],
    on="cleabs_bat",
)

# Synthese des resultats des deux methodes a l echelle des etablissements
results_comparison_schools = results_comparison_buildings.groupby("identifiant_de_l_etablissement").agg(
    surface_totale_au_sol=("surface_totale_au_sol", "sum"),  # methode simplifiée
    surface_utile=("surface_utile", "sum"),  # methode simplifiée
    potentiel_solaire=("potentiel_solaire", "sum"),  # methode simplifiée
    roof_surface=("roof_surface", "sum"),  # methode avec segmentation (MNS)
    flat_roof_surface=("flat_roof_surface", "sum"),  # methode avec segmentation (MNS)
    utility_surface=("utility_surface", "sum"),  # methode avec segmentation (MNS)
    solar_potential=("solar_potential", "sum"),  # methode avec segmentation (MNS)
).reset_index()

## Comparaisons des résultats

In [ ]:
def plot_scatter(
    x: pd.Series,
    y: pd.Series,
    title: str, 
    xlabel: str,
    ylabel: str,
    xlim: tuple = None,
    ylim: tuple = None,
    identity_line: bool = True
):
    """Plot the results of two series against each other.
    
    Args:
        x (pd.Series): The x-axis data.
        y (pd.Series): The y-axis data.
        title (str): The title of the plot.
        xlabel (str): The label for the x-axis.
        ylabel (str): The label for the y-axis.
        xlim (tuple, optional): The limits for the x-axis. Defaults to None.
        ylim (tuple, optional): The limits for the y-axis. Defaults to None.
        identity_line (bool, optional): Whether to plot the identity line. Defaults to True.
    """
    plt.rcParams['figure.figsize'] = [20, 10]

    if identity_line:
        # plot identity line
        identity_line = [0, x.max()]
        plt.plot(identity_line, identity_line, "r--")

    
    # Compute and plot linear regression line
    model = LinearRegression()
    model.fit(x.values.reshape(-1, 1), y)
    y_pred = model.predict(x.values.reshape(-1, 1))
    plt.plot(x, y_pred, "g--")
    
    # Compute and display R^2 score and model parameters
    r2 = model.score(x.values.reshape(-1, 1), y)
    slope = model.coef_[0]
    intercept = model.intercept_

    # Display R², slope, and intercept on the plot
    plt.text(
        0.05, 0.95,
        f"R²: {r2:.3f}\nSlope: {slope:.3f}\nIntercept: {intercept:.3f}",
        transform=plt.gca().transAxes,
        fontsize=14,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7)
    )
    
    # compare x and y
    plt.scatter(x, y, alpha=0.8)

    # Set the title and labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    #  Set the limits for x and y axes if provided
    if xlim:
        plt.xlim(xlim)
    
    if ylim:
        plt.ylim(ylim)

    # Show the plot 
    plt.show()

## La méthode simplifiée est-elle une bonne approximation de la surface utile ?

In [ ]:
plot_scatter(
    x=results_comparison_buildings["surface_utile"],
    y=results_comparison_buildings["utility_surface"],
    title="Surface utile (méthode simplifiée) vs surface utile des toits (méthode avec segmentation)",
    xlabel="Surface utile - méthode simplifiée (m²)",
    ylabel="Surface utile - méthode avec segmentation (m²)",
)

results_comparison_buildings["euclidean_distance"] = abs(results_comparison_buildings["surface_utile"] - results_comparison_buildings["utility_surface"])
results_comparison_buildings["euclidean_distance"].describe()

## Relation surface totale au sol <-> surface utile (via segmentation) à la maille des batiments

In [ ]:
plot_scatter(
    x=results_comparison_buildings["surface_totale_au_sol"],
    y=results_comparison_buildings["utility_surface"],
    title="Relation entre la surface totale au sol et la surface utile réelle (obtenue par segmentation des toits)",
    xlabel="Surface totale au sol (m²)",
    ylabel="Surface utile au sol (m²)",
    identity_line=False,
)

## Relation surface totale au sol <-> surface utile (via segmentation) à la maille des ecoles

In [ ]:
plot_scatter(
    x=results_comparison_schools["surface_totale_au_sol"],
    y=results_comparison_schools["utility_surface"],
    title="Relation entre la surface totale au sol et la surface utile réelle (obtenue par segmentation des toits)",
    xlabel="Surface totale au sol (m²)",
    ylabel="Surface utile au sol (m²)",
    identity_line=False,
)

In [ ]:
surface_utile_segmentation = results_comparison_schools["utility_surface"].sum()
surface_totale_au_sol = results_comparison_schools["surface_totale_au_sol"].sum()

mean_ratio = surface_utile_segmentation / surface_totale_au_sol

print(f"Surface utile totale (méthode avec segmentation) : {surface_utile_segmentation:,.0f} m²")
print(f"Surface totale au sol : {surface_totale_au_sol:,.0f} m²")
print(f"Ratio moyen surface utile / surface totale au sol : {mean_ratio:.2f}")

## L'hypothèse des toits plats est-elle pertinente ?

In [ ]:
total_flat_surface = results_with_segmentation["flat_roof_surface"].sum()
total_roof_surface = results_with_segmentation["roof_surface"].sum()
ratio_flat_roof = total_flat_surface / total_roof_surface

print(f"Surface des toits plats: {total_flat_surface} m²")
print(f"Surface totale des toits: {total_roof_surface} m²")
print(f"Ratio des toits plats: {ratio_flat_roof:.2%}")

results_with_segmentation["slope_bin_label"] = results_with_segmentation["slope_bin_min"].astype(str) + "-" + results_with_segmentation["slope_bin_max"].astype(str)
h = results_with_segmentation.groupby(by=["slope_bin_min", "slope_bin_label"])["roof_surface"].sum().reset_index().sort_values(by="slope_bin_min")

plt.rcParams['figure.figsize'] = [20, 10]
plt.bar(h["slope_bin_label"], h["roof_surface"])
plt.title("Surface des toits par pente")
plt.xlabel("Pente des toits (°)")
plt.ylabel("Surface des toits (m²)")
plt.xticks(rotation=45)

plt.show()

## Comparaison du potentiel solaire obtenu selon les deux méthodes (au niveau des établissements)

In [ ]:
plot_scatter(
    x=results_comparison_schools["potentiel_solaire"],
    y=results_comparison_schools["solar_potential"],
    title="Comparaison du potentiel solaire obtenu selon les deux méthodes (par établissement)",
    xlabel="Potentiel solaire - Méthode simplifiée (kWh/an)",
    ylabel="Potentiel solaire - Méthode avec segmentation (kWh/an)",
)

results_comparison_schools["euclidean_distance"] = abs(results_comparison_schools["potentiel_solaire"] - results_comparison_schools["solar_potential"])
results_comparison_schools["euclidean_distance"].describe()

In [ ]:
potential_methode_simplifiee = results_comparison_schools["potentiel_solaire"].sum() 
potential_methode_segmentation = results_comparison_schools["solar_potential"].sum()

print(f"Potentiel solaire total - Méthode simplifiée : {potential_methode_simplifiee:,.0f} kWh/an")
print(f"Potentiel solaire total - Méthode avec segmentation : {potential_methode_segmentation:,.0f} kWh/an")
print(f"Différence en pourcentage : {((potential_methode_simplifiee- potential_methode_segmentation) / potential_methode_segmentation) * 100:.2f}%")

# Comparaison avec le potentiel solaire des toitures (IDF)

### Identification des batiments pour lesquels on a les resulats des trois methodes (simplifiee, segmentation & IDF)

In [ ]:
# Download des resultats IDF
idf_results_path = DATA_FOLDER / "potentiel-solaire-idf.geojson"
download_file(
    url="https://hub.arcgis.com/api/v3/datasets/21e83d3c0fb3411bbc9b673afce13a1c_26/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1",
    output_filepath=idf_results_path
)
external_results_idf = gpd.read_file(idf_results_path)[["id", "st_areashape", "surf_util", "moyenne2", "production"]]
# TODO: comment savoir quelles ont ete leurs hypotheses de calculs (puissance max des panneaux installes, % de pertes, ...)

# Filtrage des resultats de nos deux methodes sur IDF 
results_comparison_buildings_idf = results_comparison_buildings[
    results_comparison_buildings["code_region"] == "11"
]
# TODO : calcul sur plus de batiments IDF avec la methode avec segmentation ?

In [ ]:
buildings_to_audit = results_comparison_buildings_idf["cleabs_bat"].unique()

# Filtrage des resultats IDF sur les batiments a auditer
external_results_idf_audits = external_results_idf[
    external_results_idf["id"].isin(buildings_to_audit)
]
print(f"{len(external_results_idf_audits)} batiments avec les resultats des trois methodes")

# Merge des resultats de toutes les methodes
all_results_idf = external_results_idf_audits.merge(
    results_comparison_buildings_idf,
    left_on='id', 
    right_on='cleabs_bat', 
    how="inner"
)

# Synthese rapide du total de chacune des trois methodes pour la surface utile
synthese_surface_utile = all_results_idf[["surface_totale_au_sol", "surf_util", "surface_utile", "utility_surface"]].sum()
print(f"Surface totale au sol (IDF) : {synthese_surface_utile['surface_totale_au_sol'] / 1_000_000:.2f} km²")
print(f"Surface utile (IDF) : {synthese_surface_utile['surf_util'] / 1_000_000:.2f} km²")
print(f"Surface utile (méthode simplifiée) : {synthese_surface_utile['surface_utile'] / 1_000_000:.2f} km²")
print(f"Surface utile (méthode avec segmentation) : {synthese_surface_utile['utility_surface'] / 1_000_000:.2f} km²")

# Synthese rapide du rapport surface utile / surface totale au sol
ratio_surface_utile_idf = synthese_surface_utile["surf_util"] / synthese_surface_utile["surface_totale_au_sol"]
print(f"Ratio surface utile / surface totale au sol (IDF) : {ratio_surface_utile_idf:.2f}")
ratio_surface_utile_idf_simplifiee = synthese_surface_utile["surface_utile"] / synthese_surface_utile["surface_totale_au_sol"]
print(f"Ratio surface utile / surface totale au sol (méthode simplifiée) : {ratio_surface_utile_idf_simplifiee:.2f}")
ratio_surface_utile_idf_segmentation = synthese_surface_utile["utility_surface"] / synthese_surface_utile["surface_totale_au_sol"]
print(f"Ratio surface utile / surface totale au sol (méthode avec segmentation) : {ratio_surface_utile_idf_segmentation:.2f}")

# Synthese rapide du total de chacune des trois methodes pour le potentiel solaire
synthese_potentiel_solaire = all_results_idf[["production", "potentiel_solaire", "solar_potential"]].sum()
print(f"Potentiel solaire (IDF) : {synthese_potentiel_solaire['production'] / 1_000_000:.2f} GWh/an")
print(f"Potentiel solaire (simplifiee) : {synthese_potentiel_solaire['potentiel_solaire'] / 1_000_000:.2f} GWh/an")
print(f"Potentiel solaire (segmentation) : {synthese_potentiel_solaire['solar_potential'] / 1_000_000:.2f} GWh/an")

### Comparaison des trois methodes sur le calcul de la surface utile

In [ ]:
plot_scatter(
    x=all_results_idf["surf_util"],
    y=all_results_idf["surface_utile"],
    title="Surface utile (IDF) vs Surface utile (Methode simplifiée)",
    xlabel="Surface utile - IDF (m²)",
    ylabel="Surface utile - Methode simplifiée (m²)"
)

all_results_idf["euclidean_distance"] = abs(all_results_idf["surf_util"] - all_results_idf["surface_utile"])
all_results_idf["euclidean_distance"].describe()

In [ ]:
plot_scatter(
    x=all_results_idf["surf_util"],
    y=all_results_idf["utility_surface"],
    title="Surface utile (IDF) vs Surface utile (Methode avec segmentation)",
    xlabel="Surface utile - IDF (m²)",
    ylabel="Surface utile - Methode avec segmentation (m²)"
)

all_results_idf["euclidean_distance"] = abs(all_results_idf["surf_util"] - all_results_idf["utility_surface"])
all_results_idf["euclidean_distance"].describe()